In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import copy


In [4]:
#this block is to define the distribution of Gaussian 
def gauss_pdf(z,mu,sigma):
    #calculate the value of probability density of variable z
    dim = z.shape[0]
    p = ((1 / (2 * np.pi)) ** (dim / 2)) * (1 / np.sqrt(np.linalg.det(sigma))) * np.exp(- 1 / 2 * np.dot(np.dot(np.transpose(z - mu),np.linalg.inv(sigma)),z - mu))
    return p

def one_d_gauss(z,mu,sigma):
    #it is used to calculate the margin probability of the joint distribution
    p = (1 / np.sqrt(2 * np.pi * sigma ** 2) * np.exp(- 1 / 2 * (z - mu) ** 2 / sigma ** 2))
    return p

In [5]:
# generate a symmetric invertible matrix, to randomly generate the sigma matrix.


def generate_random_covariance_matrix(dim):
    # Generate a random lower triangular matrix
    lower_triangular = np.tril(np.random.rand(dim, dim))
    
    # Construct the covariance matrix using Cholesky decomposition
    covariance_matrix = lower_triangular @ lower_triangular.T
    
    return covariance_matrix

# Example usage:
dimension = 3
random_cov_matrix = generate_random_covariance_matrix(dimension)
print("Random Covariance Matrix:")
print(random_cov_matrix)




Random Covariance Matrix:
[[0.29337206 0.24343119 0.43919874]
 [0.24343119 0.71650867 0.48305958]
 [0.43919874 0.48305958 0.73745456]]


In [9]:
def evaluate(sigma,mu,counts,bins):
    kl = 0
    i = 0
    delta_x = (bins[-1] - bins[0]) / (len(bins) - 1)
    bin_centers = 0.5 * (bins[1:] + bins[:-1])
    for count in counts:
        x_bin = bin_centers[i]
        i += 1
        if count == 0:
            continue
        p_i = one_d_gauss(x_bin,mu,sigma)
        kl +=  count * np.log(count / p_i)
        
    
    kl =  delta_x * kl
    return kl

In [ ]:
def proposal(Sigma,mu,i,x_set):
    dim = Sigma.shape[0]
    Sigma_pro = copy.deepcopy(Sigma)
    mu_pro = copy.deepcopy(mu)
    x_set_pro = copy.deepcopy(x_set)
    Sigma_pro[[i,0],:] = Sigma_pro[[0,i],:]
    Sigma_pro[:,[i,0]] = Sigma_pro[:,[0,i]]
    mu_pro[[i,0]] = mu_pro[[0,i]]
    x_set_pro[[i,0]] = x_set_pro[[0,i]]

    mu_a = mu_pro[[0],:]
    mu_b = mu_pro[1:,:]
    Sigma_aa = Sigma_pro[0,0]
    Sigma_bb = Sigma_pro[1:,1:]
    Sigma_ab = Sigma_pro[[0],1:]
    Sigma_ba = Sigma_pro[1:,[0]]
    x_b = x_set_pro[1:,:]
    mu_a_b = mu_a + np.dot(Sigma_ab,np.dot(np.linalg.inv(Sigma_bb),(x_b - mu_b)))
    Sigma_a_b = Sigma_aa - np.dot(Sigma_ab,np.dot(np.linalg.inv(Sigma_bb),Sigma_ba))
    mu_a_b = mu_a_b[0,0]
    if Sigma_a_b[0,0] < 0:
        print(Sigma_a_b[0,0])
    Sigma_a_b = np.sqrt(Sigma_a_b[0,0])
    
    return np.random.normal(loc = mu_a_b,scale = Sigma_a_b,size = 1)

In [ ]:
def gibbs(Sigma,mu,iteration):
    dim = mu.shape[0]
    x_set_init = np.random.rand(dim,1)
    x_set_current = x_set_init
    x_data = []

    with tqdm(total = iteration * dim) as pbar:
        for iter in range(iteration):
            for j in range(dim):
                pbar.update(1)
                xj = proposal(Sigma,mu,j,x_set_current)[0]
                x_set_current[j] = xj
                x_record = copy.deepcopy(x_set_current)
                x_data.append(x_record)


    x_data = np.reshape(np.array(x_data),(iteration * dim,dim))

    return x_data


In [ ]:
#this block is to set the init value
sample_total = 10000
dim = 10
Sigma = np.loadtxt('./p_G/10d_1000000_points_Sigma.txt')

mu = np.zeros((dim,1))
iteration = int(sample_total / dim)

start = time.time()
x_data = gibbs(Sigma,mu,iteration)
end = time.time()

In [ ]:
#data processing, draw the histgram and save infomation of time, KL to txt

KL_set = []
sample_total = iteration * dim
sample_total = 10000

for i in range(dim):
    xi = x_data[:sample_total,i]
    plt.figure()
    weights = np.ones_like(xi)/float(len(xi))
    counts,bins,patches = plt.hist(xi,bins = 50, density=True)
    x = np.linspace(mu[i,0] - 4 * np.sqrt(Sigma[i,i]),mu[i,0] + 4 * np.sqrt(Sigma[i,i]),1000)
    y = one_d_gauss(x,mu[i,0],np.sqrt(Sigma[i,i]))
    plt.plot(x,y)
    plt.title('Standard normal distribution sampling of ' + str(sample_total)+ ' points')
    plt.xlabel('x')
    plt.ylabel('counts')
    text_x = mu[i,0] - 4 * np.sqrt(Sigma[i,i])
    text_y = max(y) * 1.0
    plt.text(text_x,text_y,'mu = ' + str(round(mu[i,0],2)) + ',sigma = ' + str(round(np.sqrt(Sigma[i,i]),2)) + ',' + str(dim) +'d')
    plt.savefig('./p_G/' + str(sample_total) + '_points_' + str(dim) + 'd_slice_' + str(i) + '.png')
    counts,bins,patches = plt.hist(xi,bins = 1000,density = True)
    plt.close()
    kl = evaluate(np.sqrt(Sigma[i,i]),mu[i,0],counts,bins)
    KL_set.append(kl)

data_record = [np.array([end - start,np.max(np.array(KL_set))])]
np.savetxt('./p_G/' + str(dim) + 'd_' + str(sample_total) + '_points.txt',data_record,fmt = '%.02f')
np.savetxt('./p_G/'+ str(dim) + 'd_' + str(sample_total) + '_points_Sigma.txt',Sigma)


In [ ]:
# dim fix, sample total up, how would KL change?
dim_set = [1,2,4,8,10]
D_K = []

for dim in dim_set:
    Sigma = np.identity(dim)
    mu = np.zeros((dim,1))
    sample_total_set = [5000,10000,15000,20000,40000]
    kl_max_set = []

    for sample_total in sample_total_set:
        KL_set = []
        iteration = int(sample_total / dim)
        x_data = gibbs(Sigma,mu,iteration)
        for i in range(dim):
            xi = x_data[:,i]
            counts,bins,patches = plt.hist(xi,bins = 1000,density = True)
            kl = evaluate(np.sqrt(Sigma[i,i]),mu[i,0],counts,bins)
            KL_set.append(kl)
        kl_max_set.append(np.max(np.array(KL_set)))
    
    D_K.append(kl_max_set)

plt.figure()
for dim,kl_max_set in zip(dim_set,D_K):
    label_name = 'dim = ' + str(dim)
    plt.plot(sample_total_set,kl_max_set,marker = 'o',label = label_name)
    plt.title('The relationshiop between sample points number and KL')
    plt.xlabel('sample points number')
    plt.ylabel('KL divergence')
    plt.legend()
plt.savefig('./p_G/points_num&KL.png') 
plt.close()
